In [1]:
import os
import requests
import json
import pandas as pd
from datetime import datetime
from IPython.display import display, HTML
# MathTeachingAssistant.py has all libraries to interact with openai and to produce output files
from MathTeachingAssistant import *

Instantiate the assistant

In [2]:
assistant = MathTeachingAssistant('HIGHSCHOOL-SCOPE.xlsx','Geo-B1G-M.pdf')

In [3]:
df=assistant.preprocess_dataframe()
df

Strands  \
Week    Topic                                                                        
Week 1  The Builiding Blocks of Geometry                  GEOMETRIC REASONING (GR)   
        The Builiding Blocks of Geometry                  GEOMETRIC REASONING (GR)   
        The Builiding Blocks of Geometry                  GEOMETRIC REASONING (GR)   
Week 2  Logic and Proofs: Construction of Arguments   LOGIC & DISCRETE THEORY (LT)   
        Logic and Proofs: Construction of Arguments   LOGIC & DISCRETE THEORY (LT)   
...                                                                            ...   
Week 32 Review and Applied Problems on Trigonometry               TRIGONOMETRY (T)   
Week 33 Review and Applied Problems on Vectors       NUMBER SENSE OPERATIONS (NSO)   
        Review and Applied Problems on Vectors       NUMBER SENSE OPERATIONS (NSO)   
        Review and Applied Problems on Vectors       NUMBER SENSE OPERATIONS (NSO)   
        Review and Applied Problems on Vectors       NUMBER SENSE OPERATIONS (NSO)   

                                                                                             Standards  \
Week    Topic                                                                                            
Week 1  The Builiding Blocks of Geometry             MA.912.GR.1 \nProve and apply geometric theore...   
        The Builiding Blocks of Geometry             MA.912.GR.1 \nProve and apply geometric theore...   
        The Builiding Blocks of Geometry             MA.912.GR.1 \nProve and apply geometric theore...   
Week 2  Logic and Proofs: Construction of Arguments  MA.912.LT.4 \nDevelop an understanding of the ...   
        Logic and Proofs: Construction of Arguments  MA.912.LT.4 \nDevelop an understanding of the ...   
...                                                                                                ...   
Week 32 Review and Applied Problems on Trigonometry  MA.K12.MTR.7: Apply mathematics to real-world ...   
Week 33 Review and Applied Problems on Vectors       MA.912.NSO.3 \nRepresent and perform operation...   
        Review and Applied Problems on Vectors       MA.912.NSO.3 \nRepresent and perform operation...   
        Review and Applied Problems on Vectors       MA.912.NSO.3 \nRepresent and perform operation...   
        Review and Applied Problems on Vectors       MA.K12.MTR.7: Apply mathematics to real-world ...   

                                                    Enabling Objectives  \
Week    Topic                                                             
Week 1  The Builiding Blocks of Geometry                  MA.912.GR.5.1   
        The Builiding Blocks of Geometry                ELA.K12.EE.3.1    
        The Builiding Blocks of Geometry                 MA.K12.MTR.1.1   
Week 2  Logic and Proofs: Construction of Arguments       MA.912.LT.4.3   
        Logic and Proofs: Construction of Arguments      MA.912.LT.4.10   
...                                                                 ...   
Week 32 Review and Applied Problems on Trigonometry      MA.K12.MTR.7.1   
Week 33 Review and Applied Problems on Vectors           MA.912.NSO.3.8   
        Review and Applied Problems on Vectors           MA.912.NSO.3.6   
        Review and Applied Problems on Vectors           MA.912.NSO.3.4   
        Review and Applied Problems on Vectors           MA.K12.MTR.7.1   

                                                                                            Unnamed: 5  
Week    Topic                                                                                           
Week 1  The Builiding Blocks of Geometry                    Construct a copy of a segment or an angle.  
        The Builiding Blocks of Geometry                     Make inferences to support comprehension.  
        The Builiding Blocks of Geometry             Actively participate in effortful learning bot...  
Week 2  Logic and Proofs: Construction of Arguments  Identify and accurately

Monday.com API configuration

In [1]:
API_KEY = os.getenv('MONDAY_API_KEY', input("Enter your Monday.com API Key: "))
API_URL = "https://api.monday.com/v2"
HEADERS = {
    "Authorization": API_KEY,
    "Content-Type": "application/json"
}

NameError: name 'os' is not defined

Fetch boards

In [6]:
def get_boards():
    query = '{ boards { id name } }'
    response = requests.post(API_URL, json={'query': query}, headers=HEADERS)
    boards = response.json()['data']['boards']
    
    print("Available Boards:")
    for board in boards:
        print(f"Board ID: {board['id']}, Name: {board['name']}")
    
    return boards

Create tasks in Monday.com

In [7]:
def create_tasks(board_id, group_id,link,link_audio,text):
    created_tasks = []
    
    #for _, row in df.iterrows():
    query = '''
    mutation create_item(
        $board_id: ID!, 
        $group_id: String!, 
        $item_name: String!, 
        $column_values: JSON
    ) {
        create_item(
            board_id: $board_id,
            group_id: $group_id,
            item_name: $item_name,
            column_values: $column_values
        ) {
            id
            name
        }
    }
    '''

    column_values = json.dumps({
        'project_status': {'label': 'Working on it'},
        'date': datetime.now().strftime('%Y-%m-%d'),
        'text': link, #row.get('Link', '')
        'text4': link_audio
    })

    variables = {
        'board_id': str(board_id),
        'group_id': group_id,
        'item_name': text, #row['ProcessedTitle'],
        'column_values': column_values
    }

    response = requests.post(
        API_URL, 
        json={'query': query, 'variables': variables}, 
        headers=HEADERS
    )
    response_json = response.json()
            # Check for errors
    if 'errors' in response_json:
        print(f"Error creating task: {response_json['errors']}")
    #continue

    # Extract created item
    try:
        task = response_json['data']['create_item']
        created_tasks.append(task)
        print(f"Created task: {task['name']}")
    except KeyError:
        print(f"Unexpected response structure: {response_json}")
#         task = response.json()['data']['create_item']
#         created_tasks.append(task)
#         print(f"Created task: {task['name']}")
    
    return created_tasks

Main workflow

In [20]:
# Main workflow
def main():
    # Display and select board
    boards = get_boards()
    board_id = input("\nEnter the Board ID you want to work with: ")
    
    # Select group
    group_query = f'''
    {{
        boards(ids: {board_id}) {{
            groups {{
                id
                title
            }}
        }}
    }}
    '''
    
    groups_response = requests.post(API_URL, json={'query': group_query}, headers=HEADERS)
    groups = groups_response.json()['data']['boards'][0]['groups']
    
    print("\nAvailable Groups:")
    for group in groups:
        print(f"Group ID: {group['id']}, Name: {group['title']}")
    
    group_id = input("\nEnter the Group ID you want to create tasks in: ")
    # Process CSV and create tasks
    tsv_path = input("Enter the parent folder where your tsv located: ")
    #Process CSV file
    files=!(ls {tsv_path}/*tsv)
    dfs = [pd.read_csv(file,sep='\t') for file in files]
    ###################################
    #Submission of tasks
#     for i,(week,title) in enumerate(dict(assistant.df.index).items()):
    #Change offset according to your needs, take in mind monday doesn't allows more than ~10-12 pullings
    off=1
    for i,(week,title) in enumerate(tuple(dict(assistant.df.index).items())[off-1:off+10]):
        text = f'{week.upper()}-{title.replace(" ","-").upper()}'
        text1 = f'WORKSHOP {i+off}'
        text2 = f'WEEKLY TEST {i+off}'
        link=[list(dfs[k][dfs[k]['FILE'].str.contains(f'W-{i+off}-')]['LINK'])[0] for k in range(3)]
        create_tasks(board_id, group_id,'',link[0], text)
        create_tasks(board_id, group_id,link[2],'', text1)
        create_tasks(board_id, group_id,link[1],'', text2)

In [19]:
#4760298858
#new_group2521__1
#10th_grade/links
if __name__ == "__main__":
    main()

Available Boards:
Board ID: 6271466030, Name: My Tasks
Board ID: 4760299250, Name: Subitems of VIMS ONLINE SCHOOL
Board ID: 4760299120, Name: Project Overview
Board ID: 4760298858, Name: VIMS ONLINE SCHOOL
Board ID: 4760298702, Name: Getting Started

Enter the Board ID you want to work with: 4760298858

Available Groups:
Group ID: new_group2521__1, Name: 10TH GRADE MATH
Group ID: new_group45379__1, Name: 11TH GRADE MATH
Group ID: new_group58093__1, Name: 12TH GRADE MATH

Enter the Group ID you want to create tasks in: new_group2521__1
Enter the parent folder where your tsv located: 10th_grade/links
Created task: WEEK 23-SIMPLIFYING-WITH-TRIGONOMETRY
Created task: WORKSHOP 23
Created task: WEEKLY TEST 23
Created task: WEEK 24-THE-UNIT-CIRCLE:-TRIGONOMETRY'S-PLAYGROUND
Created task: WORKSHOP 24
Created task: WEEKLY TEST 24
Created task: WEEK 25-TRIGONOMETRIC-FUNCTIONS:-BEYOND-THE-BASICS
Created task: WORKSHOP 25
Created task: WEEKLY TEST 25
Created task: WEEK 26-INTRODUCTION-TO-VECTORS
C

In [11]:
#List of all documents by week
for i,(week,title) in enumerate(dict(assistant.df.index).items()):
#     off=23
    #for i,(week,title) in enumerate(tuple(dict(assistant.df.index).items())[off-1:off+10]):
    print(title)
    text = f'{week.upper()}-{title.replace(" ","-").upper()}'
    print(text,'\n')


The Builiding Blocks of Geometry
WEEK 1-THE-BUILIDING-BLOCKS-OF-GEOMETRY 

Logic and Proofs: Construction of Arguments
WEEK 2-LOGIC-AND-PROOFS:-CONSTRUCTION-OF-ARGUMENTS 

Unlocking the Secrets of Lines and Angles
WEEK 3-UNLOCKING-THE-SECRETS-OF-LINES-AND-ANGLES 

Triangle Mysteries: Unveiling Congruence and Similarity
WEEK 4-TRIANGLE-MYSTERIES:-UNVEILING-CONGRUENCE-AND-SIMILARITY 

Exploring Properties of Parallelograms and Trapezoids
WEEK 5-EXPLORING-PROPERTIES-OF-PARALLELOGRAMS-AND-TRAPEZOIDS 

Transformations: Changing Shapes and Perspectives
WEEK 6-TRANSFORMATIONS:-CHANGING-SHAPES-AND-PERSPECTIVES 

Mapping Figures: The Magic of Transformations
WEEK 7-MAPPING-FIGURES:-THE-MAGIC-OF-TRANSFORMATIONS 

Coordinate Geometry: The Roadmap to Geometric
WEEK 8-COORDINATE-GEOMETRY:-THE-ROADMAP-TO-GEOMETRIC 

Solving Real-World Problems with Coordinate Geometry
WEEK 9-SOLVING-REAL-WORLD-PROBLEMS-WITH-COORDINATE-GEOMETRY 

Dimensions Matter: Understanding 3D Shapes
WEEK 10-DIMENSIONS-MATTER:-U

In [12]:
pd.set_option('display.max_rows',None)
pd.set_option('display.max_columns',None)
pd.set_option('display.max_colwidth',None)

In [13]:
df#.to_string()

Strands  \
Week    Topic                                                                                    
Week 1  The Builiding Blocks of Geometry                              GEOMETRIC REASONING (GR)   
        The Builiding Blocks of Geometry                              GEOMETRIC REASONING (GR)   
        The Builiding Blocks of Geometry                              GEOMETRIC REASONING (GR)   
Week 2  Logic and Proofs: Construction of Arguments               LOGIC & DISCRETE THEORY (LT)   
        Logic and Proofs: Construction of Arguments               LOGIC & DISCRETE THEORY (LT)   
        Logic and Proofs: Construction of Arguments               LOGIC & DISCRETE THEORY (LT)   
Week 3  Unlocking the Secrets of Lines and Angles                     GEOMETRIC REASONING (GR)   
        Unlocking the Secrets of Lines and Angles                     GEOMETRIC REASONING (GR)   
        Unlocking the Secrets of Lines and Angles                     GEOMETRIC REASONING (GR)   
Week 4  Triangle Mysteries: Unveiling Congruence and Similarity       GEOMETRIC REASONING (GR)   
        Triangle Mysteries: Unveiling Congruence and Similarity       GEOMETRIC REASONING (GR)   
        Triangle Mysteries: Unveiling Congruence and Similarity       GEOMETRIC REASONING (GR)   
        Triangle Mysteries: Unveiling Congruence and Similarity       GEOMETRIC REASONING (GR)   
        Triangle Mysteries: Unveiling Congruence and Similarity       GEOMETRIC REASONING (GR)   
        Triangle Mysteries: Unveiling Congruence and Similarity       GEOMETRIC REASONING (GR)   
Week 5  Exploring Properties of Parallelograms and Trapezoids         GEOMETRIC REASONING (GR)   
        Exploring Properties of Parallelograms and Trapezoids         GEOMETRIC REASONING (GR)   
Week 6  Transformations: Changing Shapes and Perspectives             GEOMETRIC REASONING (GR)   
        Transformations: Changing Shapes and Perspectives             GEOMETRIC REASONING (GR)   
Week 7  Mapping Figures: The Magic of Transformations                 GEOMETRIC REASONING (GR)   
        Mapping Figures: The Magic of Transformations                 GEOMETRIC REASONING (GR)   
        Mapping Figures: The Magic of Transformations                 GEOMETRIC REASONING (GR)   
Week 8  Coordinate Geometry: The Roadmap to Geometric                 GEOMETRIC REASONING (GR)   
        Coordinate Geometry: The Roadmap to Geometric                 GEOMETRIC REASONING (GR)   
Week 9  Solving Real-World Problems with Coordinate Geometry          GEOMETRIC REASONING (GR)   
        Solving Real-World Problems with Coordinate Geometry          GEOMETRIC REASONING (GR)   
        Solving Real-World Problems with Coordinate Geometry          GEOMETRIC REASONING (GR)   
Week 10 Dimensions Matter: Understanding 3D Shapes                    GEOMETRIC REASONING (GR)   
        Dimensions Matter: Understanding 3D Shapes                    GEOMETRIC REASONING (GR)   
Week 11 Scale Drawings of 2D and 3D Figures                           GEOMETRIC REASONING (GR)   
        Scale Drawings of 2D and 3D Figures                           GEOMETRIC REASONING (GR)   
        Scale Drawings of 2D and 3D Figures                           GEOMETRIC REASONING (GR)   
Week 12 Volume and Surface Area: Filling and Wrapping Shapes          GEOMETRIC REASONING (GR)   
        Volume and Surface Area: Filling and Wrapping Shapes          GEOMETRIC REASONING (GR)   
Week 13 Geometric Constructions: Building with Precision              GEOMETRIC REASONING (GR)   
        Geometric Constructions: Building with Precision              GEOMETRIC REASONING (GR)   
Week 14 Circles Inside Out: Inscribed and Circumscribed               GEOMETRIC REASONING (GR)   
        Circles Inside Out: Inscribed and Circumscribed               GEOMETRIC REASONING (GR)   
Week 15 Circles and Their Properties: Lengths and Angles              GEOMETRIC REASONING (GR)   
        Circles and Their Properties: Lengths and Angles             

Preparation Notes:<br>
1. Install required libraries: pip install requests pandas<br>
2. Set MONDAY_API_KEY environment variable<br>
3. Prepare CSV with 'Title' and optional 'Link' columns